# cariGeoGAN

See below my implementation of cariGeoGAN. Again, the dataset used to train this network and generate my results is not publically avaliable. You will have to request access to the dataset: https://cs.nju.edu.cn/rl/WebCaricature.htm

If you have access to the dataset, or similar you will need to save the faces/caricature coordinates in the subdirectories faces2car/coords_A, faces2car/coords_B and the faces/caricature image datasets in the subdirectories faces2car/images_A, faces2car/images_B

<b>Training details:</b> We use the same training strategy to CycleGAN [Zhu et al. 2017a]. For all the experiments, we set $\lambda_{cyc} = 10 $  and $\lambda_{cha} = 1$ empirically and use the Adam solver [Kingma and Ba 2014] with a batch size of 1. All networks are trained from scratch with an initial learning rate of 0.0002.

In [1]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import imread as _imread
from matplotlib.pyplot import imsave as _imwrite
import cv2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import PiecewiseAffineTransform, warp
from skimage import data
from scipy.interpolate import Rbf
import cv2
#import dlib

class PointsRBF:
    def __init__(self, src, dst):
        xsrc = src[:,0]
        ysrc = src[:,1]
        xdst = dst[:,0]
        ydst = dst[:,1]
        self.rbf_x = Rbf( xsrc, ysrc, xdst)
        self.rbf_y = Rbf( xsrc, ysrc, ydst)

    def __call__(self, xy):
        x = xy[:,0]
        y = xy[:,1]
        xdst = self.rbf_x(x,y)
        ydst = self.rbf_y(x,y)
        return np.transpose( [xdst,ydst] )

def warpRBF(image, src, dst):
    prbf = PointsRBF( dst, src)
    warped = warp(image, prbf)
    warped = 255*warped                         # 0..1 => 0..255
    warped = warped.astype(np.uint8)            # convert from float64 to uint8
    return warped


def add_boundary_coords(coords):
    '''    
    takes coordinates set with shape (1, 2) and adds points at the boundary of the image
    '''
    
    # add bounary points
    coords = np.append(coords, np.array([0, 0]).reshape(1,-1), axis=0)
    coords = np.append(coords, np.array([256, 0]).reshape(1,-1), axis=0)
    coords = np.append(coords, np.array([0, 256]).reshape(1,-1), axis=0)
    coords = np.append(coords, np.array([256, 256]).reshape(1,-1), axis=0)
    
    return coords



### Utils.py

In [3]:
"""
Some codes from https://github.com/Newmu/dcgan_code
"""
from __future__ import division
import math
import pprint
import scipy.misc
import numpy as np
import copy
from skimage import transform as skimage_transform

pp = pprint.PrettyPrinter()

get_stddev = lambda x, k_h, k_w: 1/math.sqrt(k_w*k_h*x.get_shape()[-1])

# -----------------------------
# new added functions for cyclegan
class ImagePool(object):
    def __init__(self, maxsize=50):
        self.maxsize = maxsize
        self.num_img = 0
        self.images = []

    def __call__(self, image):
        if self.maxsize <= 0:
            return image
        if self.num_img < self.maxsize:
            self.images.append(image)
            self.num_img += 1
            return image
        if np.random.rand() > 0.5:
            idx = int(np.random.rand()*self.maxsize)
            tmp1 = copy.copy(self.images[idx])[0]
            self.images[idx][0] = image[0]
            idx = int(np.random.rand()*self.maxsize)
            tmp2 = copy.copy(self.images[idx])[1]
            self.images[idx][1] = image[1]
            return [tmp1, tmp2]
        else:
            return image

def load_test_data(image_path, fine_size=256):
    img = imread(image_path)
    img = skimage_transform.resize(img, (fine_size, fine_size))
    img = img/127.5 - 1
    return img

def load_train_data(image_path, load_size=286, fine_size=256, is_testing=False):
    img_A = imread(image_path[0])
    img_B = imread(image_path[1])
    if not is_testing:
        img_A = skimage_transform.resize(img_A, (load_size, load_size))
        img_B = skimage_transform.resize(img_B, (load_size, load_size))

        h1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size)))
        w1 = int(np.ceil(np.random.uniform(1e-2, load_size-fine_size)))
        img_A = img_A[h1:h1+fine_size, w1:w1+fine_size]
        img_B = img_B[h1:h1+fine_size, w1:w1+fine_size]

        if np.random.random() > 0.5:
            img_A = np.fliplr(img_A)
            img_B = np.fliplr(img_B)
    else:
        img_A = skimage_transform.resize(img_A, (fine_size, fine_size))
        img_B = skimage_transform.resize(img_B, (fine_size, fine_size))

    img_A = img_A/127.5 - 1.
    img_B = img_B/127.5 - 1.

    img_AB = np.concatenate((img_A, img_B), axis=2)
    # img_AB shape: (fine_size, fine_size, input_c_dim + output_c_dim)
    return img_AB

# -----------------------------

def get_image(image_path, image_size, is_crop=True, resize_w=64, is_grayscale = False):
    return transform(imread(image_path, is_grayscale), image_size, is_crop, resize_w)

def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)



def imread(path, is_grayscale = False):
    if (is_grayscale):
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
        return gray
    else:
        image = cv2.imread(path)
        return image

def merge_images(images, size):
    return inverse_transform(images)

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

def imsave(images, size, path):
    return _imwrite(path, merge(images, size))

def center_crop(x, crop_h, crop_w,
                resize_h=64, resize_w=64):
  if crop_w is None:
    crop_w = crop_h
  h, w = x.shape[:2]
  j = int(round((h - crop_h)/2.))
  i = int(round((w - crop_w)/2.))
  return skimage_transform.resize(
      x[j:j+crop_h, i:i+crop_w], (resize_h, resize_w))

def transform(image, npx=64, is_crop=True, resize_w=64):
    # npx : # of pixels width/height of image
    if is_crop:
        cropped_image = center_crop(image, npx, resize_w=resize_w)
    else:
        cropped_image = image
    return np.array(cropped_image)/127.5 - 1.

def inverse_transform(images):
    return (images+1.)/2.


In [4]:
# new added functions for cyclegan
class ImagePool(object):
    def __init__(self, maxsize=50):
        self.maxsize = maxsize
        self.num_img = 0
        self.images = []

    def __call__(self, image):
        if self.maxsize <= 0:
            return image
        if self.num_img < self.maxsize:
            self.images.append(image)
            self.num_img += 1
            return image
        if np.random.rand() > 0.5:
            idx = int(np.random.rand()*self.maxsize)
            tmp1 = copy.copy(self.images[idx])[0]
            self.images[idx][0] = image[0]
            idx = int(np.random.rand()*self.maxsize)
            tmp2 = copy.copy(self.images[idx])[1]
            self.images[idx][1] = image[1]
            return [tmp1, tmp2]
        else:
            return image

### ops.py

In [5]:
import math
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.framework import ops

# from utils import *

##### New Helper Functions

# weight and bais wrappers
def weight_variable(name, shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W_' + name,
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(name, shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b_' + name,
                           dtype=tf.float32,
                           initializer=initial)
 

def fc_layer(x, num_units, name, use_relu=True):
    """
    Create a fully-connected layer
    :param x: input from previous layer
    :param num_units: number of hidden units in the fully-connected layer
    :param name: layer name
    :param use_relu: boolean to add ReLU non-linearity (or not)
    :return: The output array
    """
    in_dim = x.get_shape()[1]
    W = weight_variable(name, shape=[in_dim, num_units])
    b = bias_variable(name, [num_units])
    layer = tf.matmul(x, W)
    layer += b
    if use_relu:
        layer = tf.nn.relu(layer)
    return layer


### module.py

https://www.oreilly.com/library/view/tensorflow-for-deep/9781491980446/ch04.html

In [6]:
from __future__ import division
import tensorflow as tf
# from ops import *
# from utils import *

# discriminator network
def discriminator(pca_vector, options, reuse=False, name="discriminator"):

    with tf.variable_scope(name):
        # image is 256 x 256 x input_c_dim
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse is False
            
        output_dimension = pca_vector.shape[1]

        h0 = fc_layer(pca_vector, 128, name='d_h0', use_relu=True)
        # h0 is (128 x 128 x self.df_dim)
        h1 = fc_layer(h0, 128, name='d_h1', use_relu=True)
        # h1 is (64 x 64 x self.df_dim*2)
        h2 = fc_layer(h1, 128, name='d_h2', use_relu=True)
        # h2 is (32x 32 x self.df_dim*4)
        h3 = fc_layer(h2, 128, name='d_h3', use_relu=True)
        # h3 is (32 x 32 x self.df_dim*8)
        h4 = fc_layer(h3, output_dimension, name='d_h4', use_relu=True)
        # h4 is (32 x 32 x 1)
        return h4

# generator network without residual block
def generator(pca_vector, options, reuse=False, name="generator"):

    with tf.variable_scope(name):
        # pca_vector is ((k=32 < m=number of coordinates x 2) x input_c_dim)
        if reuse:
            tf.get_variable_scope().reuse_variables()
        else:
            assert tf.get_variable_scope().reuse is False
            
        output_dimension = pca_vector.shape[1]

        # pca_vector is ((k=32 < m=number of coordinates x 2) x input_c_dim)
        e1 = fc_layer(pca_vector, 64, name='g_e1', use_relu=True)
        # e1 is (128 x 128 x self.gf_dim) 1048576
        e2 = fc_layer(e1, 128, name='g_e2', use_relu=True)
        # e2 is (64 x 64 x self.gf_dim*2) 524288
        e3 = fc_layer(e2, 256, name='g_e3', use_relu=True)
        # e3 is (32 x 32 x self.gf_dim*4) 262144
        e4 = fc_layer(e3, 512, name='g_e4', use_relu=True)
        # e4 is (16 x 16 x self.gf_dim*8) 131072
        e5 = fc_layer(e4, 256, name='g_e5', use_relu=True)
        # e5 is (8 x 8 x self.gf_dim*8) 32768
        e6 = fc_layer(e5, 128, name='g_e6', use_relu=True)
        # e6 is (4 x 4 x self.gf_dim*8) 8192
        e7 = fc_layer(e6, 64, name='g_e7', use_relu=True)
        # e7 is (2 x 2 x self.gf_dim*8) 2048
        e8 = fc_layer(e7, output_dimension, name='g_e8', use_relu=False)
        # e8 is (1 x 1 x self.gf_dim*8) 512

        return e8

    
# loss function
def abs_criterion(in_, target):
    return tf.reduce_mean(tf.abs(in_ - target))

## loss function
def mae_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)

## loss function
def sce_criterion(logits, labels):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))


## loss function
def cosine_difference(real_X, real_X_mean, fake_Y, real_Y_mean):

    # real_y minus mean of real_Y for all y in Y
    A = real_X - real_X_mean

    # fake_x minus mean of X for all y in Y
    B = fake_Y - real_Y_mean
    
    normalize_A = tf.nn.l2_normalize(A,1)        
    normalize_B = tf.nn.l2_normalize(B,1)
    
    difference_matrix = 1 - tf.matmul(normalize_A, normalize_B, transpose_b=True)

    difference_matrix = tf.diag_part(difference_matrix)
    
    difference = tf.reduce_sum(difference_matrix)
    
    return difference
    
    


### Model.py

In [46]:
from __future__ import division
import os
import time
from glob import glob
import tensorflow as tf
import numpy as np
from collections import namedtuple


# cyclegan class created with build_model, train, save, load, sample_model, new and test methods
class cariGeoGAN(object):
    def __init__(self, sess, args):
        # initialise tensorflow session
        self.sess = sess
        
        # batch size
        self.batch_size = args.batch_size
        
        # data, test, train splits
        df_A = pd.read_csv('coords_A.csv', index_col=0)
        df_B = pd.read_csv('coords_B.csv', index_col=0)
        self.df_A_train, self.df_A_test = train_test_split(df_A, test_size=100, random_state=0)
        self.df_B_train, self.df_B_test = train_test_split(df_B, test_size=100, random_state=0)
        
        
        # number of principle components 
        self.num_components = args.pca_components
        
        # standard scaler and pca
        self.sc_A = StandardScaler()
        self.sc_B = StandardScaler()
        self.pca_A = PCA(n_components=self.num_components)
        self.pca_B = PCA(n_components=self.num_components)
        
        # create pca data frames, feature scaling,  principle component analysis
        self.df_pca_A_train = pd.DataFrame(self.pca_A.fit_transform(self.sc_A.fit_transform(self.df_A_train)), 
                                           list(self.df_A_train.index))
        self.df_pca_B_train = pd.DataFrame(self.pca_B.fit_transform(self.sc_B.fit_transform(self.df_B_train)), 
                                           list(self.df_B_train.index))

        self.df_pca_A_test = pd.DataFrame(self.pca_A.transform(self.sc_A.transform(self.df_A_test)), 
                                          list(self.df_A_test.index))
        self.df_pca_B_test = pd.DataFrame(self.pca_B.transform(self.sc_B.transform(self.df_B_test)), 
                                          list(self.df_B_test.index))
        

        
        # lambda1 and lambda2 balance the cycle consistancy loss and characteristic (respetively)
        self.L1_lambda = args.L1_lambda
        self.L2_lambda = args.L2_lambda
        
        # directory of dataset
        self.dataset_dir = args.dataset_dir
        
        # discriminator and generator networks
        self.discriminator = discriminator
        self.generator = generator

        # choice of cost function for advasarial loss
        if args.use_lsgan:
            self.criterionGAN = mae_criterion
        else:
            self.criterionGAN = sce_criterion

        # options for various functions throughout the class
        OPTIONS = namedtuple('OPTIONS', 'batch_size image_size \
                              gf_dim df_dim is_training')
        self.options = OPTIONS._make((args.batch_size, args.pca_components,
                                      args.ngf, args.ndf,
                                      args.phase == 'train'))
        
        # when an instance of class cycleGAN is created, build model is automatically called
        self._build_model()
        
        # saver
        self.saver = tf.train.Saver()
        

    def _build_model(self):
        
        #### INPUTS TO NETWORKS
        # placeholder for real data, each dataSet represents one of two ends of the cycle e.g. faces vs cariCatures
        self.real_pca_A = tf.placeholder(tf.float32,
                                     [None, self.num_components],
                                     name='real_A')
        self.real_pca_B = tf.placeholder(tf.float32,
                                    [None, self.num_components],
                                    name='real_B')
        self.real_pca_mean_A = tf.placeholder(tf.float32,
                                     [None, self.num_components],
                                     name='real_A')
        self.real_pca_mean_B = tf.placeholder(tf.float32,
                                    [None, self.num_components],
                                    name='real_B')
        
        #### GENERATOR NETWORKS
        # create two generators: inputA->outputB and inputB->outputA (generator cycle)
        # generate fakeB from realA (G)
        # generate reconstructedA (comparable to realA) from fakeB (F)
        # generate fakeA from realB (F)
        # generate reconstructedB (comparable to realB) from fakeA (G)
        self.fake_pca_B = self.generator(self.real_pca_A, self.options, False, name="generatorA2B")
        self.fake_pca_A_ = self.generator(self.fake_pca_B, self.options, False, name="generatorB2A")
        self.fake_pca_A = self.generator(self.real_pca_B, self.options, True, name="generatorB2A")
        self.fake_pca_B_ = self.generator(self.fake_pca_A, self.options, True, name="generatorA2B")
        
        #### DISCRIMINATOR NETWORKS
        # IAIN: create two discriminators networks
        # IAIN: discriminate, takes fake_B as input from generator and outputs DB_fake (0-1) 0 = fake, 1 = not_fake
        # IAIN: discriminate, takes fake_A as input from generator and outputs DA_fake (0-1) 0 = fake, 1 = not_fake
        self.DB_pca_fake = self.discriminator(self.fake_pca_B, self.options, reuse=False, name="discriminatorB")
        self.DA_pca_fake = self.discriminator(self.fake_pca_A, self.options, reuse=False, name="discriminatorA")

        #### GENERATOR LOSS FUNCTION
        # IAIN: definine total loss functions for generator
        # IAIN: three loss functions definied
        # IAIN: g_loss_a2b and g_loss_b2a are minimised as g_loss is minismised. although they do not impact the network
        # in any way, they are recorded so they can be observed independantly in the tf.summary
        # IAIN: g_loss is used to optimise the generator network and is made up of four components
        # IAIN: adversarial loss: self.criterionGAN(self.DA_pca_fake, tf.ones_like(self.DA_pca_fake))
        # IAIN: adversarial loss: self.criterionGAN(self.DB_pca_fake, tf.ones_like(self.DB_pca_fake))
        # IAIN: cycle consistancy loss: self.L1_lambda * abs_criterion(self.real_pca_A, self.fake_pca_A_)
        # IAIN: cycle consistancy loss: self.L1_lambda * abs_criterion(self.real_pca_B, self.fake_pca_B_)
        # IAIN: characteristic loss: self.char_loss_a2b
        # IAIN: characteristic loss: self.char_loss_b2a
        self.g_loss_a2b = self.criterionGAN(self.DB_pca_fake, tf.ones_like(self.DB_pca_fake)) \
            + self.L1_lambda * abs_criterion(self.real_pca_A, self.fake_pca_A_) \
            + self.L1_lambda * abs_criterion(self.real_pca_B, self.fake_pca_B_)
        self.g_loss_b2a = self.criterionGAN(self.DA_pca_fake, tf.ones_like(self.DA_pca_fake)) \
            + self.L1_lambda * abs_criterion(self.real_pca_A, self.fake_pca_A_) \
            + self.L1_lambda * abs_criterion(self.real_pca_B, self.fake_pca_B_)
        
        self.char_loss_a2b = self.L2_lambda * cosine_difference(self.real_pca_A, self.real_pca_mean_A, 
                                                                self.fake_pca_B, self.real_pca_mean_B)
        self.char_loss_b2a = self.L2_lambda * cosine_difference(self.real_pca_B, self.real_pca_mean_B, 
                                                                self.fake_pca_A, self.real_pca_mean_A)
        
        
        self.g_loss = self.criterionGAN(self.DA_pca_fake, tf.ones_like(self.DA_pca_fake)) \
            + self.criterionGAN(self.DB_pca_fake, tf.ones_like(self.DB_pca_fake)) \
            + self.L1_lambda * abs_criterion(self.real_pca_A, self.fake_pca_A_) \
            + self.L1_lambda * abs_criterion(self.real_pca_B, self.fake_pca_B_) \
            + self.L2_lambda * cosine_difference(self.real_pca_A, self.real_pca_mean_A, 
                                                                self.fake_pca_B, self.real_pca_mean_B) \
            + self.L2_lambda * cosine_difference(self.real_pca_B, self.real_pca_mean_B, 
                                                                self.fake_pca_A, self.real_pca_mean_A)
        
        
        #### INPUTS FOR DISCRIMINATOR NETOWRK
        # IAIN: tf.plaeholders for fake_image data created by generator networks
        # IAIN: in train function of class you can see fake_A_sample/fake_B_sample 
        # defined as output of generator network
        ##### NOTE!!!!!!! check vs dimension in original construction encase of error included extra dimension info
        self.fake_pca_A_sample = tf.placeholder(tf.float32,
                                            [None, self.num_components], name='fake_A_sample')
        self.fake_pca_B_sample = tf.placeholder(tf.float32,
                                            [None, self.num_components], name='fake_B_sample')
        
        #### DISCRIMINATOR NETWORKS CONTINUED
        # IAIN: define 4 outputs from the two discriminator networks
        # IAIN: all four outputs fed to discriminator loss function
        # IAIN: discriminate, takes real_B as input (raw data) and outputs DB_real (0-1) 0 = ???, 1 = ???
        # IAIN: discriminate, takes real_A as input (raw data) and outputs DA_real (0-1) 0 = ???, 1 = ???
        # IAIN: discriminate, takes fake_B_sample as input (raw data) and outputs DB_fake_sample (0-1) 0 = ???, 1 = ???
        # IAIN: discriminate, takes fake_A_sample as input (raw data) and outputs DA_fake_sample (0-1) 0 = ???, 1 = ???
        # IAIN: reuse = TRUE, essentially this reuses the variables from previous discriminator network with the same name i.e. //
        # IAIN: // the same discriminator network. The outputs can then be used within the objective functions to optimise that disctiminator. 
        self.DB_pca_real = self.discriminator(self.real_pca_B, self.options, reuse=True, name="discriminatorB")
        self.DA_pca_real = self.discriminator(self.real_pca_A, self.options, reuse=True, name="discriminatorA")
        self.DB_pca_fake_sample = self.discriminator(self.fake_pca_B_sample, self.options, reuse=True, name="discriminatorB")
        self.DA_pca_fake_sample = self.discriminator(self.fake_pca_A_sample, self.options, reuse=True, name="discriminatorA")
        
        #### DISCRIMINATOR NETWORKS
        # IAIN: discriminator loss function (all variables below help define d_loss)
        # IAIN: advasarial loss only
        self.db_loss_real = self.criterionGAN(self.DB_pca_real, tf.ones_like(self.DB_pca_real))
        self.db_loss_fake = self.criterionGAN(self.DB_pca_fake_sample, tf.zeros_like(self.DB_pca_fake_sample))
        self.db_loss = (self.db_loss_real + self.db_loss_fake) / 2
        self.da_loss_real = self.criterionGAN(self.DA_pca_real, tf.ones_like(self.DA_pca_real))
        self.da_loss_fake = self.criterionGAN(self.DA_pca_fake_sample, tf.zeros_like(self.DA_pca_fake_sample))
        self.da_loss = (self.da_loss_real + self.da_loss_fake) / 2
        self.d_loss = self.da_loss + self.db_loss

        #### CREATE SUMMARY
        # tf.summary is a tensorflow module to allow you to track variables over time visually
        self.g_loss_a2b_sum = tf.summary.scalar("g_loss_a2b", self.g_loss_a2b)
        self.g_loss_b2a_sum = tf.summary.scalar("g_loss_b2a", self.g_loss_b2a)
        self.char_loss_a2b_sum = tf.summary.scalar("g_char_loss_a2b", self.char_loss_a2b)
        self.char_loss_b2a_sum = tf.summary.scalar("g_char_loss_b2a", self.char_loss_b2a)
        self.g_loss_sum = tf.summary.scalar("g_loss", self.g_loss)
        
        # tf.summary.merge: https://www.tensorflow.org/api_docs/python/tf/summary/merge
        # is a 'protocol buffer' serializing structured data – think XML but smaller, faster, and simpler
        self.g_sum = tf.summary.merge([self.g_loss_a2b_sum, self.g_loss_b2a_sum, self.char_loss_a2b_sum,
                                       self.char_loss_b2a_sum, self.g_loss_sum])

        # tf.summary is a tensorflow module to allow you to track variables over time visually
        self.db_loss_sum = tf.summary.scalar("db_loss", self.db_loss)
        self.da_loss_sum = tf.summary.scalar("da_loss", self.da_loss)
        self.d_loss_sum = tf.summary.scalar("d_loss", self.d_loss)
        self.db_loss_real_sum = tf.summary.scalar("db_loss_real", self.db_loss_real)
        self.db_loss_fake_sum = tf.summary.scalar("db_loss_fake", self.db_loss_fake)
        self.da_loss_real_sum = tf.summary.scalar("da_loss_real", self.da_loss_real)
        self.da_loss_fake_sum = tf.summary.scalar("da_loss_fake", self.da_loss_fake)

        # tf.summary.merge: https://www.tensorflow.org/api_docs/python/tf/summary/merge
        # is a 'protocol buffer' serializing structured data – think XML but smaller, faster, and simpler
        self.d_sum = tf.summary.merge(
            [self.da_loss_sum, self.da_loss_real_sum, self.da_loss_fake_sum,
             self.db_loss_sum, self.db_loss_real_sum, self.db_loss_fake_sum,
             self.d_loss_sum])

        
        #### TESTING
        # IAIN: defines placeholders for test input data
        # IAIN: defines testB and testA as output of generator functions for testing
        self.test_A = tf.placeholder(tf.float32,
                                     [None, self.num_components], name='test_A')
        self.test_B = tf.placeholder(tf.float32,
                                     [None, self.num_components], name='test_B')
        self.testB = self.generator(self.test_A, self.options, True, name="generatorA2B")
        self.testA = self.generator(self.test_B, self.options, True, name="generatorB2A")

        t_vars = tf.trainable_variables()

        
        #### TRAINING
        # IAIN: d_vars and g_vars used in train function
        # IAIN: variables needed for AdamOptimizer
        # IAIN: used in AdamOptimizer in conjunction with d_loss and g_loss
        self.d_vars = [var for var in t_vars if 'discriminator' in var.name]
        self.g_vars = [var for var in t_vars if 'generator' in var.name]
        for var in t_vars: print(var.name)


    def train(self, args):
        """Train cyclegan"""
        # placeholder for learning rate
        self.lr = tf.placeholder(tf.float32, None, name='learning_rate')

        # define optimizers for d_loss and g_loss
        self.d_optim = tf.train.AdamOptimizer(self.lr, beta1=args.beta1) \
            .minimize(self.d_loss, var_list=self.d_vars)
        self.g_optim = tf.train.AdamOptimizer(self.lr, beta1=args.beta1) \
            .minimize(self.g_loss, var_list=self.g_vars)

        # initialise global varibles and run session
        init_op = tf.global_variables_initializer()
        self.sess.run(init_op)

        # log writer
        self.writer = tf.summary.FileWriter("./logs", self.sess.graph)

        # track progress
        counter = 1
        start_time = time.time()

        # train from checkpoint rather than form scratch
        if args.continue_train:
            if self.load(args.checkpoint_dir):
                print(" [*] Load SUCCESS")
            else:
                print(" [!] Load failed...")
                
        #
        np_A_pca = self.df_pca_A_train.copy().to_numpy()
        np_B_pca = self.df_pca_B_train.copy().to_numpy()
        np_A_pca_mean = np_A_pca.mean(axis=0).reshape(1,-1)
        np_B_pca_mean = np_B_pca.mean(axis=0).reshape(1,-1)

        # iterate over the number of epochs definied
        for epoch in range(args.epoch):

            # IAIN: create stochastic / batch ???
            
            ### IMOPRT DATA
            # import data
            df_A_pca = self.df_pca_A_train.copy()
            df_B_pca = self.df_pca_B_train.copy()

            # 
            file_names_A = list(df_A_pca.index)
            file_names_B = list(df_B_pca.index)
            
            # shuffle order of list
            np.random.shuffle(file_names_A)
            np.random.shuffle(file_names_B)
            
            ##################################
                        
            # train size very large, batch size = 1, so this is just min of len(dataA) and len(dataB)
            batch_idxs = min(min(len(file_names_A), len(file_names_B)), args.train_size) // self.batch_size
            
            # the learning rate is kept equal for first 100 epochs and then linearly decays over the next 100
            lr = args.lr if epoch < args.epoch_step else args.lr*(args.epoch-epoch)/(args.epoch-args.epoch_step)            
            
            
            ##################################
            #  
            for idx in range(0, batch_idxs):
                
                # load one image from each dataset
                data_pca_A = df_A_pca.loc[file_names_A[idx * self.batch_size:(idx + 1) * self.batch_size]].to_numpy().astype(np.float32)
                data_pca_B = df_B_pca.loc[file_names_B[idx * self.batch_size:(idx + 1) * self.batch_size]].to_numpy().astype(np.float32)
                
                
                ##################################
                
                
                # cast images astype float32
                # batch_images = np.array(batch_images).astype(np.float32)

                # Update G network and record fake outputs
                # input: real images A and B and learning rate
                # output: fake images from generator networks and cost / summaries
                fake_pca_A, fake_pca_B, _, summary_str = self.sess.run(
                    [self.fake_pca_A, self.fake_pca_B, self.g_optim, self.g_sum],
                    feed_dict={self.real_pca_A: data_pca_A, 
                               self.real_pca_B: data_pca_B,
                               self.real_pca_mean_A: np_A_pca_mean, 
                               self.real_pca_mean_B: np_B_pca_mean,
                               self.lr: lr})
                self.writer.add_summary(summary_str, counter)
                #[fake_A, fake_B] = self.pool([fake_A, fake_B])
                


                # Update D network
                # input: real / fake images and learning rates
                # output: cost / summaries
                _, summary_str = self.sess.run(
                    [self.d_optim, self.d_sum],
                    feed_dict={self.real_pca_A: data_pca_A,
                               self.real_pca_B: data_pca_A,
                               self.fake_pca_A_sample: fake_pca_A,
                               self.fake_pca_B_sample: fake_pca_B,
                               self.lr: lr})
                self.writer.add_summary(summary_str, counter)
                
                # track progress: print epoch, idx, batch_idxs and runtime
                counter += 1
                if counter % 5000 == 0:
                    print(("Epoch: [%2d] [%4d/%4d] time: %4.4f" % (
                        epoch, idx, batch_idxs, time.time() - start_time)))

                # IAIN: calls class method sample_model
                # IAIN: ??? periodically save (and record??) sample model
                if epoch in [0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190] and counter == 5000:
                    self.sample_model(args.sample_dir, epoch, idx)
                

                # calls class method save
                # IAIN: ???? periodically save sample model
                if np.mod(counter, args.save_freq) == 2:
                    self.save(args.checkpoint_dir, counter)
                    
  

    def save(self, checkpoint_dir, step):
        # save model
        model_name = "cariGeoGAN.model"
        model_dir = "%s_%s" % (self.dataset_dir, self.num_components)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,
                        os.path.join(checkpoint_dir, model_name),
                        global_step=step)

    def load(self, checkpoint_dir):
        #load model
        print(" [*] Reading checkpoint...")

        model_dir = "%s_%s" % (self.dataset_dir, self.num_components)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            return True
        else:
            return False

    def sample_model(self, sample_dir, epoch, idx):
        
        ### gather batch of test images
        df_A_pca = self.df_pca_A_test.copy()
        df_B_pca = self.df_pca_B_test.copy()
        
        # ...
        file_names_A = list(df_A_pca.index)
        file_names_B = list(df_B_pca.index)
        
        
        # shuffle order of list
        np.random.shuffle(file_names_A)
        np.random.shuffle(file_names_B)

        # file
        files_A = np.random.choice(file_names_A, self.batch_size)
        files_B = np.random.choice(file_names_B, self.batch_size)
        
        # 
        data_A = df_A_pca.loc[files_A].to_numpy().astype(np.float32)
        data_B = df_B_pca.loc[files_B].to_numpy().astype(np.float32)             
        
        ### feed test images into network and return fake output images
        fake_pca_A, fake_pca_B = self.sess.run(
            [self.fake_pca_A, self.fake_pca_B],
            feed_dict={self.real_pca_A: data_A,
                       self.real_pca_B: data_B}
        )
        
        ### reverse pca / reverse scaling
        fake_A_recon = self.sc_A.inverse_transform(self.pca_A.inverse_transform(fake_pca_A))
        fake_B_recon = self.sc_B.inverse_transform(self.pca_B.inverse_transform(fake_pca_B))
        
        ### from and to coordintes
        coords_from_A = self.df_A_test.loc[files_A].to_numpy().reshape(20,2)
        coords_to_A = fake_B_recon.reshape(20,2)
        coords_from_B = self.df_B_test.loc[files_B].to_numpy().reshape(20,2) 
        coords_to_B = fake_A_recon.reshape(20,2)
        
        # cocatenate boundary points  
        coords_from_A = add_boundary_coords(coords_from_A)
        coords_to_A = add_boundary_coords(coords_to_A)
        coords_from_B = add_boundary_coords(coords_from_B)
        coords_to_B = add_boundary_coords(coords_to_B)

        
        ### original imges
        image_A = imread('datasets/faces2cari/images_A/' + files_A[0], is_grayscale = False)
        image_B = imread('datasets/faces2cari/images_B/' + files_B[0], is_grayscale = False)
        
        
        ### load and warp image
        fake_image_A = warpRBF(image_A, coords_from_A, coords_to_A)
        fake_image_B = warpRBF(image_B, coords_from_B, coords_to_B)
        
        
        # summary_image
        summary_image_A = cv2.hconcat([image_A, fake_image_A])
        summary_image_B = cv2.hconcat([image_B, fake_image_B])
        
        #
        cv2.imwrite('sample/' + files_A[0], summary_image_A)
        cv2.imwrite('sample/' + files_B[0], summary_image_B)

    def test(self, args):
        """Test cyclegan"""
        init_op = tf.global_variables_initializer()
        self.sess.run(init_op)
        
        # choose direction for text
        if args.which_direction == 'AtoB':
            df_test = self.df_A_test.copy()
            df_test_pca = self.df_pca_A_test.copy()
            sample_files = list(self.df_pca_A_test.index)
            directory = 'datasets/faces2cari/images_A/'
        elif args.which_direction == 'BtoA':
            df_test = self.df_B_test.copy()
            df_test_pca = self.df_pca_B_test.copy()
            sample_files = list(self.df_pca_B_test.index)
            directory = 'datasets/faces2cari/images_B/'
        else:
            raise Exception('--which_direction must be AtoB or BtoA')
        
        # load checkpoint
        if self.load(args.checkpoint_dir):
            print(" [*] Load SUCCESS")
        else:
            print(" [!] Load failed...")

        # write html for visual comparison
        index_path = os.path.join(args.test_dir, '{0}_index.html'.format(args.which_direction))
        index = open(index_path, "w")
        index.write("<html><body><table><tr>")
        index.write("<th>name</th><th>input</th><th>output</th></tr>")
        
        
        # direction testing
        out_var, in_var, out_pca, out_sc = (self.testB, self.test_A, self.pca_B, self.sc_B) if args.which_direction == 'AtoB' else (
            self.testA, self.test_B, self.pca_A, self.sc_A)

        
        for sample_file in sample_files:

            image = imread(directory + sample_file, is_grayscale = False)
        
            data = df_test_pca.loc[sample_file].to_numpy().astype(np.float32).reshape(1,-1)
        
            ### feed test images into network and return fake output images
            fake_pca = self.sess.run(out_var, feed_dict={in_var: data})

            ### reverse pca / reverse scaling
            fake_recon = out_sc.inverse_transform(out_pca.inverse_transform(fake_pca))

            ### from and to coordintes
            coords_from = df_test.loc[sample_file].to_numpy().reshape(20,2)
            coords_to = fake_recon.reshape(20,2)

            # cocatenate boundary points  
            coords_from = add_boundary_coords(coords_from)
            coords_to = add_boundary_coords(coords_to)


            ### load and warp image
            fake_image = warpRBF(image, coords_from, coords_to)
            
            # summary_image
            summary_image = cv2.hconcat([image, fake_image])
       

            ### save images in sample model directory
            cv2.imwrite('test/' + sample_file, summary_image)
            
    def new(self, args, image, coords):
        """Test cyclegan"""
        init_op = tf.global_variables_initializer()
        self.sess.run(init_op)
                
        # load checkpoint
        if self.load(args.checkpoint_dir):
            print(" [*] Load SUCCESS")
        else:
            print(" [!] Load failed...") 
            
        
        # coords_pca
        coords_pca = self.pca_A.transform(self.sc_A.transform(coords))
        
        
        # direction testing
        out_var, in_var, out_pca, out_sc = (self.testB, self.test_A, self.pca_B, self.sc_B) if args.which_direction == 'AtoB' else (
            self.testA, self.test_B, self.pca_A, self.sc_A)

        ### feed test images into network and return fake output images
        fake_pca = self.sess.run(out_var, feed_dict={in_var: coords_pca})

        ### reverse pca / reverse scaling
        fake_recon = out_sc.inverse_transform(out_pca.inverse_transform(fake_pca))

        ### from and to coordintes
        coords_from = coords.reshape(20,2)
        coords_to = fake_recon.reshape(20,2)

        # cocatenate boundary points  
        coords_from = add_boundary_coords(coords_from)
        coords_to = add_boundary_coords(coords_to)


        ### load and warp image
        fake_image = warpRBF(image, coords_from, coords_to)

        # summary_image
        summary_image = cv2.hconcat([image, fake_image])


        ### save images in sample model directory
        cv2.imwrite('new/' + 'new_cari.jpg', summary_image)



### Train Model

In [39]:
class Args():
    dataset_dir = 'faces2cari'
    epoch = 200
    epoch_step = 100
    batch_size = 1
    train_size = 1e8
    load_size = 286
    pca_components = 32
    ngf = 64
    ndf = 64
    lr = 0.0002
    beta1 = 0.5
    which_direction = 'AtoB'
    phase = 'test'
    save_freq = 5000
    print_freq = 100
    continue_train = False
    checkpoint_dir = './checkpoint'
    sample_dir = './sample'
    test_dir = './test'
    L1_lambda = 10.0
    L2_lambda = 1.0
    use_lsgan = True
    max_size = 50

args = Args()

In [12]:
# TRAIN
tf.reset_default_graph()

tfconfig = tf.ConfigProto(allow_soft_placement=True)
tfconfig.gpu_options.allow_growth = True
with tf.Session(config=tfconfig) as sess:
    model = cariGeoGAN(sess, args)
    model.train(args) 

generatorA2B/W_g_e1:0
generatorA2B/b_g_e1:0
generatorA2B/W_g_e2:0
generatorA2B/b_g_e2:0
generatorA2B/W_g_e3:0
generatorA2B/b_g_e3:0
generatorA2B/W_g_e4:0
generatorA2B/b_g_e4:0
generatorA2B/W_g_e5:0
generatorA2B/b_g_e5:0
generatorA2B/W_g_e6:0
generatorA2B/b_g_e6:0
generatorA2B/W_g_e7:0
generatorA2B/b_g_e7:0
generatorA2B/W_g_e8:0
generatorA2B/b_g_e8:0
generatorB2A/W_g_e1:0
generatorB2A/b_g_e1:0
generatorB2A/W_g_e2:0
generatorB2A/b_g_e2:0
generatorB2A/W_g_e3:0
generatorB2A/b_g_e3:0
generatorB2A/W_g_e4:0
generatorB2A/b_g_e4:0
generatorB2A/W_g_e5:0
generatorB2A/b_g_e5:0
generatorB2A/W_g_e6:0
generatorB2A/b_g_e6:0
generatorB2A/W_g_e7:0
generatorB2A/b_g_e7:0
generatorB2A/W_g_e8:0
generatorB2A/b_g_e8:0
discriminatorB/W_d_h0:0
discriminatorB/b_d_h0:0
discriminatorB/W_d_h1:0
discriminatorB/b_d_h1:0
discriminatorB/W_d_h2:0
discriminatorB/b_d_h2:0
discriminatorB/W_d_h3:0
discriminatorB/b_d_h3:0
discriminatorB/W_d_h4:0
discriminatorB/b_d_h4:0
discriminatorA/W_d_h0:0
discriminatorA/b_d_h0:0
discrimi

Epoch: [149] [4772/5874] time: 9029.6634
Epoch: [150] [3898/5874] time: 9084.2228
Epoch: [151] [3024/5874] time: 9139.2001
Epoch: [152] [2150/5874] time: 9194.7084
Epoch: [153] [1276/5874] time: 9248.9081
Epoch: [154] [ 402/5874] time: 9303.2621
Epoch: [154] [5402/5874] time: 9358.1980
Epoch: [155] [4528/5874] time: 9412.7964
Epoch: [156] [3654/5874] time: 9469.2563
Epoch: [157] [2780/5874] time: 9524.1969
Epoch: [158] [1906/5874] time: 9578.7634
Epoch: [159] [1032/5874] time: 9633.9845
Epoch: [160] [ 158/5874] time: 9689.9565
Epoch: [160] [5158/5874] time: 9743.8126
Epoch: [161] [4284/5874] time: 9792.6768
Epoch: [162] [3410/5874] time: 9842.1262
Epoch: [163] [2536/5874] time: 9896.8680
Epoch: [164] [1662/5874] time: 9951.9655
Epoch: [165] [ 788/5874] time: 10006.6087
Epoch: [165] [5788/5874] time: 10061.9642
Epoch: [166] [4914/5874] time: 10117.9825
Epoch: [167] [4040/5874] time: 10173.3206
Epoch: [168] [3166/5874] time: 10229.2104
Epoch: [169] [2292/5874] time: 10283.9174
Epoch: [17

In [21]:
# RESTORE
sess.close()
tf.reset_default_graph()

### Test Model

In [12]:
run_test = True

if run_test:
    # TEST
    tfconfig = tf.ConfigProto(allow_soft_placement=True)
    tfconfig.gpu_options.allow_growth = True
    with tf.Session(config=tfconfig) as sess:
        model_restored = cariGeoGAN(sess, args)
        model_restored.test(args)
    

generatorA2B/W_g_e1:0
generatorA2B/b_g_e1:0
generatorA2B/W_g_e2:0
generatorA2B/b_g_e2:0
generatorA2B/W_g_e3:0
generatorA2B/b_g_e3:0
generatorA2B/W_g_e4:0
generatorA2B/b_g_e4:0
generatorA2B/W_g_e5:0
generatorA2B/b_g_e5:0
generatorA2B/W_g_e6:0
generatorA2B/b_g_e6:0
generatorA2B/W_g_e7:0
generatorA2B/b_g_e7:0
generatorA2B/W_g_e8:0
generatorA2B/b_g_e8:0
generatorB2A/W_g_e1:0
generatorB2A/b_g_e1:0
generatorB2A/W_g_e2:0
generatorB2A/b_g_e2:0
generatorB2A/W_g_e3:0
generatorB2A/b_g_e3:0
generatorB2A/W_g_e4:0
generatorB2A/b_g_e4:0
generatorB2A/W_g_e5:0
generatorB2A/b_g_e5:0
generatorB2A/W_g_e6:0
generatorB2A/b_g_e6:0
generatorB2A/W_g_e7:0
generatorB2A/b_g_e7:0
generatorB2A/W_g_e8:0
generatorB2A/b_g_e8:0
discriminatorB/W_d_h0:0
discriminatorB/b_d_h0:0
discriminatorB/W_d_h1:0
discriminatorB/b_d_h1:0
discriminatorB/W_d_h2:0
discriminatorB/b_d_h2:0
discriminatorB/W_d_h3:0
discriminatorB/b_d_h3:0
discriminatorB/W_d_h4:0
discriminatorB/b_d_h4:0
discriminatorA/W_d_h0:0
discriminatorA/b_d_h0:0
discrimi

In [47]:
# RESTORE
sess.close()
tf.reset_default_graph()

### New Example

In [48]:
run_new = True
image = cv2.imread('new/new.jpg')
coords_df = pd.read_csv('new/new_coords.csv', index_col=0)
coords = coords_df.to_numpy()

if run_new:
    # TEST
    tfconfig = tf.ConfigProto(allow_soft_placement=True)
    tfconfig.gpu_options.allow_growth = True
    with tf.Session(config=tfconfig) as sess:
        model_restored = cariGeoGAN(sess, args)
        model_restored.new(args, image, coords)
    

generatorA2B/W_g_e1:0
generatorA2B/b_g_e1:0
generatorA2B/W_g_e2:0
generatorA2B/b_g_e2:0
generatorA2B/W_g_e3:0
generatorA2B/b_g_e3:0
generatorA2B/W_g_e4:0
generatorA2B/b_g_e4:0
generatorA2B/W_g_e5:0
generatorA2B/b_g_e5:0
generatorA2B/W_g_e6:0
generatorA2B/b_g_e6:0
generatorA2B/W_g_e7:0
generatorA2B/b_g_e7:0
generatorA2B/W_g_e8:0
generatorA2B/b_g_e8:0
generatorB2A/W_g_e1:0
generatorB2A/b_g_e1:0
generatorB2A/W_g_e2:0
generatorB2A/b_g_e2:0
generatorB2A/W_g_e3:0
generatorB2A/b_g_e3:0
generatorB2A/W_g_e4:0
generatorB2A/b_g_e4:0
generatorB2A/W_g_e5:0
generatorB2A/b_g_e5:0
generatorB2A/W_g_e6:0
generatorB2A/b_g_e6:0
generatorB2A/W_g_e7:0
generatorB2A/b_g_e7:0
generatorB2A/W_g_e8:0
generatorB2A/b_g_e8:0
discriminatorB/W_d_h0:0
discriminatorB/b_d_h0:0
discriminatorB/W_d_h1:0
discriminatorB/b_d_h1:0
discriminatorB/W_d_h2:0
discriminatorB/b_d_h2:0
discriminatorB/W_d_h3:0
discriminatorB/b_d_h3:0
discriminatorB/W_d_h4:0
discriminatorB/b_d_h4:0
discriminatorA/W_d_h0:0
discriminatorA/b_d_h0:0
discrimi